# Tutorial for ibuffpy

We demonstrate how to use ibuffpy in several examples. The theoretical backgrounds are described in our paper.

**Yuhei Yamauchi, Atsuki Hishida, Takashi Okada, and Atsushi Mochizuki, Finding regulatory modules of chemical reaction systems, bioRxiv 2023**

To install ibuffpy, run the following command:

$ pip install ibuffpy

In [1]:
from ibuffpy import ReactionNetwork
import importlib
importlib.reload(ReactionNetwork)

<module 'ibuffpy.ReactionNetwork' from '/opt/anaconda3/lib/python3.8/site-packages/ibuffpy/ReactionNetwork.py'>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout

## Example 1

We consider a chemical reaction network (CRN), as shown below (Yamauchi et al., 2023 Figs. 1 and 2). Nodes represent chemicals, and arrows indicate chemical reactions.

<img src="./figure/example1.png" width="500"> 

The information of the CRN is summarizes in "example1.csv".

In [3]:
crn_pd_example1= pd.read_csv ("example1.csv")
crn_pd_example1

,Reaction_index,Substrate,Product,Activator,Inhibitor
0,1,NaN,P,NaN,NaN
1,2,P,Q,NaN,NaN
2,3,Q,R,NaN,NaN
3,4,R,NaN,NaN,NaN


Each row in the dataframe corresponds to each reaction.<br>
The first column ("Reaction_id") represents the reaction ID.<br>
The second column ("Substrate") and the third column ("Product") represents the substrate and the product of the reaction, respectively.<br>
The fourth ("Activator") and fifth column ("Inhibitor") represents the set of chemicals that regulate the flux of the reaction positively or negatively, respectively. <br>

If there is no chemical in a cell of the dataframe, "np.nan" is placed. If there are multiple chemicals in a cell, use some seperators such as space (defalut) or comma (,).

First, we construct a new instance of the ReactionNetwork class. If you start from pandas dataframe, the function ReactionNetwork.from_pandas should be used.

In [18]:
# construct a class
network_example1 = ReactionNetwork.from_pandas(crn_pd_example1, ker_basis='svd', cq_basis='rref', sep = ",")
#sep = "," means chemicals in each cell in the pandas dataframe are seperated by comma (by default, sep = " "(space)).

constructed.
out node
M =  3
R =  4


The argument ker_basis shows how to calculate the basis for the nullspace of $\mathrm{ker} \hspace{2pt} \boldsymbol{\nu}$ (cycles), where $\boldsymbol{\nu}$ is the stoichiometric matrix.<br>
The argument cq_basis show how to calculate the basis for the nullspace of $\mathrm{ker} \hspace{2pt} \boldsymbol{\nu}^{\top}$ (conserved quantities).<br>

If 'svd', nullspace vectors are calculated via singular value decomposition (SVD). 
If 'rref', RREF is used.
You can use basis of your own making (should be the row vectors).


In general, 'svd' is faster than 'rref', but results in non-sparse vectors.
For ker_basis, the choice of the basis does not affect the result, hence 'svd' is recommended.
For cq_basis, the choice of the basis can affect the result, and 'rref' is recommended.

The chemicals and reactions are

In [5]:
print(f'The number of chemicals is {network_example1.M}.')
print(f'The number of reactions is {network_example1.R}.')
print(f'The set of chemicals is {network_example1.cpd_list_noout}.')
print(f'The set of reactions is {network_example1.reactionNames}.')

The number of chemicals is 3.
The number of reactions is 4.
The set of chemicals is ['P', 'Q', 'R'].
The set of reactions is ['1', '2', '3', '4'].


The stoichiometric matrix is obtained by self.stoi.
A stoichiometry matrix is an M-by-R matrix, where M equals the total number of species in a model, and R equals the total number of reactions in a model. Each row corresponds to a species, and each column corresponds to a reaction.

In [21]:
stoi1=network_example1.stoi#show the stoichiometric matrix using pandas.
pd.DataFrame (stoi1, columns = network_example1.reactionNames, index = network_example1.cpd_list_noout)

,1,2,3,4
P,1.0,-1.0,0.0,0.0
Q,0.0,1.0,-1.0,0.0
R,0.0,0.0,1.0,-1.0


For example, from the first colum, you can see that when the reaction 1 proceed at one time, the number of the chemical P increase by one, but the number of the chemical Q or R is not changed.
Similary, from the second colum, you can see that when the reaction 2 proceed at one time, the number of the chemical P and Q decrease and oncrease by one, respectively.<br>

The basis for cycles and the conserved quantites are given by self.ns and self.ns2, respectively. Both are given by column vector.

In [22]:
print(f'Cycles \n{network_example1.ns}')
print(f'Conserved quantities\n{network_example1.ns2}')#No conserved quantity

Cycles 
[[0.5]
 [0.5]
 [0.5]
 [0.5]]
Conserved quantities
[]


### Finding buffering structure

Buffering structures are identified using the function ReactionNetwork.compute_bs (self), which returns a list of buffering structures.

In [23]:
#Find buffering structures
bs_example1 = ReactionNetwork.compute_bs(network_example1)
print (f'Buffering structures\n{bs_example1}')

Buffering structures
[[['P'], ['2']], [['Q'], ['3']], [['R'], ['4']], [['P', 'Q', 'R'], ['1', '2', '3', '4']]]


Each buffering structure is represented by the list of list (The first element= the set of chemicals, the second element = the set of reactions).

From this result, all buffering structures are as follows.

<img src="./figure/example1_bs.png" width="500"> 

You can obtain a hierarchy graph, which visualizes the regulatory patterns of the network.

In [24]:
# draw a hierarchy
#pygraphviz is required
hier_agraph_example1 = ReactionNetwork.make_hiergraph(bs_example1)
hier_agraph_example1.draw(f'./figure/hier_example1.png')

The hierarchy graph is

<img src="./figure/hier_example1.png" width="300"> 

Each node plus its downstream nodes corresponds to a buffering structure.<br>

The hierarchy graph also summarizes the nonzero response pattern, i.e., modulating the enzyme activity of reactions within a square box can lead to nonzero responses in the chemicals within that box and those in the lower boxes, while leaving the other chemicals unaffected.<br>

For example, from the hierarchy graph, you can see that the perturbation to the reaction rate parameter of the reaction 1 affects P, Q, and R, since these chemcials are located downstream of the box containing the reaction 1.
Similary, the perturbation to the reaction rate parameter of the reaction 2 affects only P, since P is included in the box containing the reaction 1, but the box containing Q or R are not located downstream of the box containing the reaction 1.

### Calculating sensitivity

Our algorithm to find buffering structures are based on structural sensitivity analysis (SSA).

This analysis allows us to determine qualitative changes in the steady-state concentration of each chemical in response to changes in a reaction rate parameter (Mochizuki and Fidler,. Journal of theoretical biology 2015, Okada and Mochizuki,. Physical review letters 2016).

The SSA is done by the method compute_smat_mean. In this method, random values are assigned to $r_{j,m} \neq 0$ in the matirx $\boldsymbol{A}$  (see Section 2 of Yamauchi et al., 2023) and the $\boldsymbol{S}\coloneqq -\boldsymbol{A}^{-1}$ are caluclated numerically.

$\boldsymbol{S}$ is called a sensitivity matrix.
We repeat this for N times (10 times by defalult) and the mean of $\boldsymbol{S}$ is calculated. Using this mean, we determine the zero distribution of $\boldsymbol{S}$.

In [25]:
sensitivity_mat= network_example1.compute_smat_mean(N=10)
#Display the sensitivity matrix by using pandas.
sensitivity_mat=pd.DataFrame (sensitivity_mat[:network_example1.M], columns = network_example1.reactionNames, index = network_example1.cpd_list_noout)
sensitivity_mat.applymap (lambda x: 0 if np.abs (x)<1e-10 else 1)

,1,2,3,4
P,1,1,0,0
Q,1,0,1,0
R,1,0,0,1


The signs of changes in the steady-state concentraions of each chemical can also be determined from network topology.
"compute_smat_sign" method can be used for this purpose.

In [26]:
sensitivity_mat_sign=network_example1.compute_smat_sign()
pd.DataFrame (sensitivity_mat_sign[:network_example1.M], columns = network_example1.reactionNames, index = network_example1.cpd_list_noout)

,1,2,3,4
P,+,-,0,0
Q,+,0,-,0
R,+,0,0,-


For example, from the second column, we can see that the increase in the reaction rate parameter of the reaction 2 results in a decrease in the steady-state concentration of the chemical P, but the steady-state concentrations of either Q or R are not affected.

## Example 2

We consider a CRN with 6 chemicals (A,B,C,D,E,F) and 8 reactions ($1,\ldots,8$) (Yamauchi et al., Fig. 2)

<img src="./figure/example2.png" width="400"> 

Finding buffering structures and depicting the hierarchy graph cab be done in the same way as Example 1.

In [12]:
crn_pd_example2= pd.read_csv ("example2.csv") 
crn_pd_example2

,Reaction_index,Substrate,Product,Activator,Inhibitor
0,R1,NaN,A,NaN,NaN
1,R2,A,B,NaN,NaN
2,R3,B,C,NaN,NaN
3,R4,D,A,NaN,NaN
4,R5,E,B,NaN,NaN
5,R6,"C,E",F,NaN,NaN
6,R7,D,E,NaN,NaN
7,R8,F,NaN,NaN,NaN


In [13]:
# construct a class
network_example2 = ReactionNetwork.from_pandas(crn_pd_example2, sep = ",")#ker_basis='svd', cq_basis='rref' by defalult
print(f'Cycles\n{network_example2.ns}')
print(f'Conserved quantities\n{network_example2.ns2}')#No conserved quantity

#Find buffering structures
bs_example2 = ReactionNetwork.compute_bs(network_example2)
print (f'Buffering structures\n{bs_example2}')

# draw a hierarchy
#pygraphviz is required
hier_agraph_example2 = ReactionNetwork.make_hiergraph(bs_example2)
hier_agraph_example2.draw(f'./figure/hier_example2.png')

constructed.
out node
M =  6
R =  8
Cycles
[[ 0.59558453  0.23565967]
 [ 0.5562021  -0.31764073]
 [ 0.29779226  0.11782984]
 [-0.03938243 -0.5533004 ]
 [-0.25840984  0.43547056]
 [ 0.29779226  0.11782984]
 [ 0.03938243  0.5533004 ]
 [ 0.29779226  0.11782984]]
Conserved quantities
[]
Buffering structures
[[['A'], ['R2']], [['B'], ['R3']], [['C'], ['R6']], [['F'], ['R8']], [['C', 'E'], ['R5', 'R6']], [['A', 'C', 'D', 'E'], ['R2', 'R4', 'R5', 'R6', 'R7']], [['A', 'B', 'C', 'E', 'F'], ['R1', 'R2', 'R3', 'R5', 'R6', 'R8']]]


The hierarchy graph is

<img src="./figure/hier_example2.png" width="300"> 

## Example 3

We consider a CRN comprising six chemicals (A,B,C,D,E,F) and six reactions (1,2,3,4,5,6). Solid lines indicate chemical reactions, while the dashed line indicates active regulation (Yamauchi et al., Fig. S2).

<img src="./figure/example3.png" width="400"> 

In [14]:
crn_pd_example3= pd.read_csv ("example3.csv") 
crn_pd_example3

,Reaction_index,Substrate,Product,Activator,Inhibitor
0,R1,A,B,NaN,NaN
1,R2,B,A,NaN,NaN
2,R3,C,D,B,NaN
3,R4,D,C,NaN,NaN
4,R5,E,F,D,NaN
5,R6,F,E,NaN,NaN


In [15]:
# construct a class
network_example3 = ReactionNetwork.from_pandas(crn_pd_example3, sep = ",")#ker_basis='svd', coker_basis='rref' by defalult
print(f'Cycles \n{network_example3.ns}')
print(f'Conserved quantities\n {network_example3.ns2}')#No conserved quantity

constructed.
no out node
M =  6
R =  6
Cycles 
[[0.70710678 0.         0.        ]
 [0.70710678 0.         0.        ]
 [0.         0.70710678 0.        ]
 [0.         0.70710678 0.        ]
 [0.         0.         0.70710678]
 [0.         0.         0.70710678]]
Conserved quantities
 [[1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]]


Unlike Example 1 or Example 2, this CRN has conserved quantities ($x_A+x_B$, $x_C+x_D$, and $x_E+x_F$).
In the presence of conserved quantities, steady-state concentrations are affected not only by reaction rate parameters but also by the initial values of conserved quantities. Therefore, in this case, there are two types of perturbations; the perturbation of the reaction rate parameter and that of the conserved quantity.

When the CRN has conserved quantities, choosing the cokernel basis requires caution.
If the cokernel basis are chosen such that no two cokernel vectors share the support, such basis is suitable for finding buffering structures (see Proposition S2 in Yamauchi et al., bioRxiv 2023).

In this example, no two conserved quantities (network_example3.ns2) share support, which is suitable for finding buffering structures.

In [16]:
#Find buffering structures
bs_example3 = ReactionNetwork.compute_bs(network_example3)
print (f'Buffering structures\n{bs_example3}')

# draw a hierarchy
#pygraphviz is required
hier_agraph_example3 = ReactionNetwork.make_hiergraph(bs_example3)
hier_agraph_example3.draw(f'./figure/hier_example3.png')

# draw a hierarchy
#pygraphviz is required
hier_agraph_example3 = ReactionNetwork.make_hiergraph(bs_example3)
hier_agraph_example3.draw(f'./figure/hier_example3.png')

Buffering structures
[[['E', 'F'], ['R5', 'R6', 'cons_2']], [['C', 'D', 'E', 'F'], ['R3', 'R4', 'R5', 'R6', 'cons_1', 'cons_2']], [['A', 'B', 'C', 'D', 'E', 'F'], ['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'cons_0', 'cons_1', 'cons_2']]]


The hierarchy graph is 

<img src="./figure/hier_example3.png" width="200"> 